# Lesson 4: Creating Relationships Between Models

# Connecting Models Together

Welcome back! After learning how to integrate SQLite3 with the Django ORM and create a simple model, it's time to extend those capabilities. In this unit, we'll focus on creating relationships between models. Just like in the real world, data often has connections and relationships that need to be represented within your database. By the end of this unit, you'll be comfortable defining these relationships in Django.

---

## What You'll Learn

In this lesson, you’ll master the following:

- 📦 **Creating models** that have relationships with each other
- 🔗 **Connecting models** using Django ORM
- 💻 **Creating views** to handle data interactions involving related models

---

### Defining Models with Relationships

Let's consider a `Category` model that can be linked to multiple `Todo` items. This will allow each task to belong to a specific category.

#### models.py

```python
from django.db import models

# Category model to hold category names
class Category(models.Model):
    name = models.CharField(max_length=100)

    def __str__(self):
        return self.name

# Todo model with a ForeignKey to Category
class Todo(models.Model):
    task = models.CharField(max_length=200)
    category = models.ForeignKey(Category, on_delete=models.CASCADE)

    def __str__(self):
        return self.task
```

Here, the `Category` model holds category names, and the `Todo` model has a foreign key connecting each task to a category. 

- **Foreign keys** are used to establish relationships between models.
- The `on_delete=models.CASCADE` argument ensures that when a category is deleted, all tasks linked to that category will also be deleted.

---

### Creating Views to Handle Data

Next, we’ll create views to handle adding new categories and tasks with their corresponding categories.

#### views.py

```python
import json
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Category, Todo

# View to add a new category
@csrf_exempt
def add_category(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        new_category = Category(name=data['name'])
        new_category.save()
        return JsonResponse({'id': new_category.id, 'name': new_category.name}, status=201)
    return JsonResponse({'message': 'Invalid request'}, status=400)

# View to add a new todo with a category
@csrf_exempt
def add_todo_with_category(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        category = Category.objects.get(name=data['category'])
        new_todo = Todo(task=data['task'], category=category)
        new_todo.save()
        return JsonResponse({'id': new_todo.id, 'task': new_todo.task, 'category': new_todo.category.name}, status=201)
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

These views will enable you to add new categories and assign tasks to them through API requests.

---

### Mapping Views to URLs

Don’t forget to map these views in your `urls.py` file:

#### urls.py

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('add-category/', views.add_category, name='add_category'),
    path('add-todo-with-category/', views.add_todo_with_category, name='add_todo_with_category'),
]
```

This configuration sets up the URL routes to handle the creation of categories and tasks linked to them.

---

### Testing the Endpoints

1. **Create a Category**

Send a POST request to the `/add-category/` endpoint with the following JSON payload:

```json
{
    "name": "Work"
}
```

2. **Create a Task Linked to the Category**

Send a POST request to the `/add-todo-with-category/` endpoint with the following JSON payload:

```json
{
    "task": "Prepare presentation",
    "category": "Work"
}
```

After sending these requests, you should see the new category and task added to the database.

---

## Why It Matters

Understanding how to create and manage relationships between models is crucial for building robust and well-structured applications. With these skills, you will be able to:

- 🔗 Represent complex data relationships in your applications
- 🛠 Simplify data retrieval and manipulation using Django’s ORM
- 🚀 Build powerful and scalable web applications

By the end of this lesson, you will have a solid grasp of how to define and manage relationships between different parts of your data. This knowledge will form the backbone of efficient database management in your Django projects.

---

Are you excited to put this new knowledge into practice? Let's dive into the practice section and start coding!

## Creating Relationships Between Models

This example demonstrates how to link a `Category` model with a `Todo` model using a foreign key in Django. Let me walk you through how the code works:

### 1. **Models (Category and Todo)**

```python
from django.db import models

class Category(models.Model):
    name = models.CharField(max_length=100)

    def __str__(self):
        return self.name

class Todo(models.Model):
    task = models.CharField(max_length=200)
    category = models.ForeignKey(Category, on_delete=models.CASCADE)

    def __str__(self):
        return self.task
```

In this code:
- A `Category` model with just a `name` field is created.
- A `Todo` model is created with a `task` field and a foreign key `category` that links it to the `Category` model.
- The `on_delete=models.CASCADE` ensures that when a category is deleted, all related `Todo` entries are also deleted.

### 2. **Views (Handling Requests)**

```python
import json
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Category, Todo

@csrf_exempt
def add_category(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        new_category = Category(name=data['name'])
        new_category.save()
        return JsonResponse({'id': new_category.id, 'name': new_category.name}, status=201)
    return JsonResponse({'message': 'Invalid request'}, status=400)

@csrf_exempt
def add_todo_with_category(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        category = Category.objects.get(name=data['category'])
        new_todo = Todo(task=data['task'], category=category)
        new_todo.save()
        return JsonResponse({'id': new_todo.id, 'task': new_todo.task, 'category': new_todo.category.name}, status=201)
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

- `add_category`: This view handles adding a new category by taking a POST request with `name` in the request body.
- `add_todo_with_category`: This view handles adding a new todo item. It retrieves the category by its name and associates it with the new todo.

### 3. **Sending HTTP Requests**

```python
import requests

URL = 'http://127.0.0.1:3000'

# Add a new category
response = requests.post(URL + '/add-category/', json={'name': 'Groceries'})
print(response.json())

# Add a new todo with the 'Groceries' category
response = requests.post(URL + '/add-todo-with-category/', json={'category': 'Groceries', 'task': 'Buy milk'})
print(response.json())
```

Here, two HTTP POST requests are made:
1. To add a new category ("Groceries").
2. To add a new task ("Buy milk") under the "Groceries" category.

### 4. **URL Configuration**

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('add-category/', views.add_category, name='add_category'),
    path('add-todo-with-category/', views.add_todo_with_category, name='add_todo_with_category'),
]
```

This sets up the URL paths for both the `add_category` and `add_todo_with_category` views.

---

### Running the Code

1. **Migrations**: Before running this, make sure to create and apply migrations to reflect these models in the database:
   ```
   python manage.py makemigrations
   python manage.py migrate
   ```

2. **Start the Django server**:
   ```
   python manage.py runserver
   ```

3. **Run the `send_request.py` file** to observe the interaction with the Django app. It will:
   - First create a new category ("Groceries").
   - Then create a new todo task ("Buy milk") linked to the "Groceries" category.

You should see the responses for each action printed in the terminal.

## Add a Priority Model

To modify the behavior so that a new category isn't created every time we add a todo, we need to check if the category already exists in the database and reuse it if it does. This way, we'll only create a new category if one doesn't exist.

### Updated Code

#### 1. **View: Modify `add_todo_with_category` to check for existing categories**

```python
import json
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Category, Todo

@csrf_exempt
def add_category(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        new_category = Category(name=data['name'])
        new_category.save()
        return JsonResponse({'id': new_category.id, 'name': new_category.name}, status=201)
    return JsonResponse({'message': 'Invalid request'}, status=400)

@csrf_exempt
def add_todo_with_category(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        # Change: Try to get an existing category by name, or create it if it doesn't exist
        category, created = Category.objects.get_or_create(name=data['category'])
        
        # Create a new todo and link it to the category
        new_todo = Todo(task=data['task'], category=category)
        new_todo.save()

        return JsonResponse({'id': new_todo.id, 'task': new_todo.task, 'category': new_todo.category.name}, status=201)
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

#### 2. **Explanation of Changes**
- The key change is using `Category.objects.get_or_create()`. This method:
  - Tries to get an existing category based on the provided name.
  - If the category does not exist, it creates a new one.
- We also check if the category was newly created, but we do not necessarily need to use the `created` variable here. The method will handle category reuse effectively.

#### 3. **Test the Updated Code**

The rest of the code, including the `models.py`, `send_request.py`, and `urls.py`, remains the same. However, the key difference is that now the app will reuse existing categories instead of creating a new one each time.

Here’s the `send_request.py` for reference:

```python
import requests

URL = 'http://127.0.0.1:3000'

# Add a new category
response = requests.post(URL + '/add-category/', json={'name': 'Groceries'})
print(response.json())

# Add a new todo with the 'Groceries' category
response = requests.post(URL + '/add-todo-with-category/', json={'category': 'Groceries', 'task': 'Buy milk'})
print(response.json())
```

#### 4. **Running the Code**
1. Start the Django server:
   ```
   python manage.py runserver
   ```

2. Run `send_request.py`. You will:
   - First create a "Groceries" category.
   - Then add a "Buy milk" task under the "Groceries" category. This time, the app will check if the category already exists instead of creating a new one.

### Expected Output

For the first request (category creation):
```json
{
    "id": 1,
    "name": "Groceries"
}
```

For the second request (todo creation):
```json
{
    "id": 2,
    "task": "Buy milk",
    "category": "Groceries"
}
```

If you try to add another task under the same category, it will not create a new "Groceries" category but reuse the existing one.

## Fix the Relationships in Models

The bug in the models definition lies in the `Todo` model. The `category` field is defined as a foreign key to the `Todo` model instead of the `Category` model. This is incorrect because the `Todo` model should reference the `Category` model to establish the foreign key relationship.

### Here's the corrected models definition:

```python
from django.db import models

class Category(models.Model):
    name = models.CharField(max_length=100)

    def __str__(self):
        return self.name

class Todo(models.Model):
    task = models.CharField(max_length=200)
    category = models.ForeignKey(Category, on_delete=models.CASCADE)  # Corrected the foreign key reference

    def __str__(self):
        return self.task
```

### Explanation of the Fix:
- The `category` field in the `Todo` model should reference the `Category` model, not the `Todo` model. By changing `category = models.ForeignKey(Todo, on_delete=models.CASCADE)` to `category = models.ForeignKey(Category, on_delete=models.CASCADE)`, we correctly establish the foreign key relationship to the `Category` model.

With this change, the models now properly represent the relationship where each `Todo` is linked to a `Category`.

### Next Steps:
1. After making this change, make sure to run migrations to update your database schema:
   ```bash
   python manage.py makemigrations
   python manage.py migrate
   ```

2. Then, you can proceed with testing the API to ensure that todos are correctly associated with categories.

Let me know if you run into any issues!

## Creating Song Model

Let's complete the implementation for both the `Song` model and the `add_song_with_band` view function based on the TODO comments.

### 1. Completing the `Song` model:
We need to add two fields:
- `title`: A `CharField` with a max length of 200.
- `band`: A `ForeignKey` linking to the `Band` model, with `on_delete` set to `CASCADE`.

### 2. Completing the `add_song_with_band` view:
We will:
- Retrieve the `Band` object from the database using the name provided in the request data.
- Create a new `Song` object and associate it with the retrieved `Band`.
- Save the `Song` to the database and return a JSON response with the `Song` and `Band` details.

### Here’s the complete code:

#### Models (`models.py`):

```python
from django.db import models

class Band(models.Model):
    name = models.CharField(max_length=100)

    def __str__(self):
        return self.name

class Song(models.Model):
    title = models.CharField(max_length=200)  # Title field for the song
    band = models.ForeignKey(Band, on_delete=models.CASCADE)  # ForeignKey linking to Band

    def __str__(self):
        return self.title
```

#### Views (`views.py`):

```python
import json
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Band, Song

@csrf_exempt
def add_band(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        new_band = Band(name=data['name'])
        new_band.save()
        return JsonResponse({'id': new_band.id, 'name': new_band.name}, status=201)
    return JsonResponse({'message': 'Invalid request'}, status=400)

@csrf_exempt
def add_song_with_band(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        # Retrieve the Band object using the band name from the request data
        try:
            band = Band.objects.get(name=data['band'])
        except Band.DoesNotExist:
            return JsonResponse({'message': 'Band not found'}, status=404)

        # Create a new Song object and save it to the database
        new_song = Song(title=data['title'], band=band)
        new_song.save()

        # Return the new song's details
        return JsonResponse({
            'id': new_song.id, 
            'title': new_song.title, 
            'band': new_song.band.name
        }, status=201)

    return JsonResponse({'message': 'Invalid request'}, status=400)
```

#### Requests (`send_request.py`):

```python
import requests

URL = 'http://127.0.0.1:3000'

# Add a band
response = requests.post(URL + '/add-band/', json={'name': 'The Beatles'})
print(response.json())

# Add a song associated with the band
response = requests.post(URL + '/add-song/', json={'band': 'The Beatles', 'title': 'Hey Jude'})
print(response.json())
```

#### URL Configuration (`urls.py`):

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('add-band/', views.add_band, name='add_band'),
    path('add-song/', views.add_song_with_band, name='add_song_with_band'),
]
```

### Explanation:
1. **Model Changes**:
   - We added a `title` field for `Song`, which holds the song's name.
   - We linked `Song` to `Band` using a `ForeignKey`, ensuring each song is associated with a specific band. The `on_delete=models.CASCADE` ensures that if a band is deleted, all its songs are deleted too.

2. **View Function**:
   - In `add_song_with_band`, we retrieve the `Band` using the `Band.objects.get()` method. If the band is not found, a `404` error is returned.
   - Once the band is retrieved, we create a new `Song` object and associate it with the retrieved `Band`, then save it to the database.
   - Finally, we return the song's details, including its title and the band's name, in the response.

3. **Testing**:
   - The `send_request.py` file sends requests to add a new band and then a song linked to that band. You should be able to see the correct responses with the newly created band and song details.

### Next Steps:
- Run the migrations to create the new fields in the database:
   ```bash
   python manage.py makemigrations
   python manage.py migrate
   ```

- Test the requests and ensure everything works as expected.

## Creating Relationships Between Models

Let's break this down and complete the tasks:

### 1. **Completing the `Book` Model**:
- We need to define a `ForeignKey` relationship between `Book` and `Author` in the `Book` model. We'll set `on_delete=models.CASCADE`, meaning that when an `Author` is deleted, all their books will also be deleted.

### 2. **Completing the Views**:
- `add_author`: This function should parse the request data, create a new `Author`, and return a JSON response with the newly created author's details.
- `add_book_with_author`: This function should retrieve the `Author` based on the name provided, create a `Book` with the retrieved author, and return the `Book` details.

### Complete Code:

#### Models (`models.py`):

```python
from django.db import models

class Author(models.Model):
    name = models.CharField(max_length=100)

    def __str__(self):
        return self.name

class Book(models.Model):
    title = models.CharField(max_length=200)
    author = models.ForeignKey(Author, on_delete=models.CASCADE)  # ForeignKey linking to Author

    def __str__(self):
        return self.title
```

#### Views (`views.py`):

```python
import json
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Author, Book

@csrf_exempt
def add_author(request):
    if request.method == 'POST':
        # Parse the request body to get the author's name
        data = json.loads(request.body)
        author_name = data.get('name')

        # Create a new Author object and save it to the database
        new_author = Author(name=author_name)
        new_author.save()

        # Return a JSON response with the author's id and name
        return JsonResponse({'id': new_author.id, 'name': new_author.name}, status=201)
    
    return JsonResponse({'message': 'Invalid request'}, status=400)

@csrf_exempt
def add_book_with_author(request):
    if request.method == 'POST':
        # Parse the request body to get the author's name and book title
        data = json.loads(request.body)
        author_name = data.get('author')
        book_title = data.get('title')

        # Get the Author object from the database using the author's name
        try:
            author = Author.objects.get(name=author_name)
        except Author.DoesNotExist:
            return JsonResponse({'message': 'Author not found'}, status=404)

        # Create a new Book object with the title and author and save it to the database
        new_book = Book(title=book_title, author=author)
        new_book.save()

        # Return a JSON response with the book's id, title, and author's name
        return JsonResponse({
            'id': new_book.id, 
            'title': new_book.title, 
            'author': new_book.author.name
        }, status=201)
    
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

#### Testing the API with Requests (`send_request.py`):

```python
import requests

URL = 'http://127.0.0.1:3000'

# Add an author
response = requests.post(URL + '/add-author/', json={'name': 'J.K. Rowling'})
print(response.json())

# Add a book associated with the author
response = requests.post(URL + '/add-book/', json={'author': 'J.K. Rowling', 'title': 'Harry Potter'})
print(response.json())
```

#### URL Configuration (`urls.py`):

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('add-author/', views.add_author, name='add_author'),
    path('add-book/', views.add_book_with_author, name='add_book_with_author'),
]
```

### Explanation:

1. **Model Changes**:
   - The `Book` model has a `ForeignKey` relationship with the `Author` model. This links each book to an author. When an author is deleted, their books are also deleted (`on_delete=models.CASCADE`).

2. **View Functions**:
   - `add_author`: This view function creates an `Author` object and returns a JSON response with the new author's details.
   - `add_book_with_author`: This view function finds the `Author` by name. If found, it creates a new `Book` associated with the `Author` and returns the book's details. If the author doesn't exist, a `404` error is returned.

3. **Testing**:
   - The `send_request.py` file sends requests to add a new author and then adds a book associated with that author.

### Steps to Run:
1. Make sure you've applied migrations to create the new fields in the database:
   ```bash
   python manage.py makemigrations
   python manage.py migrate
   ```

2. Test the API requests by running the `send_request.py` file or sending the requests via Postman.

Everything should now work as expected, and you will be able to create authors and associate books with them!